In [5]:
import numpy as np
import math
from qutip import create, basis, sigmaz, sigmax, sigmay

In [12]:
omega_r = 2*np.pi*0.80
kai = -2*np.pi*0.18
qubit_initial = 0
cavity_initial = 1
superposition= False
planck = 1.0545718*10**(-34)
omega_r = omega_r
kai = kai
kappa = 2*np.pi*(7.2) #MHz * time = t
# dt = 1e-3 : 1e-9 actual
# times = n * dt
n_cavity = 10
ad = create(n_cavity)
a = ad.dag()

qubit_initial = qubit_initial
cavity_initial = cavity_initial
superposition = superposition

# Hamiltonian
h_int = np.kron(kai*0.5*(ad*a), sigmaz())
h_r = np.kron(np.identity(n_cavity),(0.5)*omega_r*sigmax())
h = h_int + h_r

# initial state
if qubit_initial ==0:
    qubit_state = basis(2, 1)
else:
    qubit_state = basis(2, 0)

if superposition == True:
    cavity_state = (basis(n_cavity, cavity_initial) + basis(n_cavity,cavity_initial+1))/np.sqrt(2)
else:
    cavity_state = basis(n_cavity, cavity_initial)

psi0 = np.kron(cavity_state, qubit_state)

def exp_H(t):
    from scipy import linalg
    return linalg.expm(-1j*h*t)

def bra_m(c, q):
    if q == 0:
        q_state = 1
    else:
        q_state = 0
        
    return np.kron(basis(n_cavity, c), basis(2, q_state))
    
def prob(c, q,t):
    return (abs(np.dot(bra_m(c, q).T, np.dot(exp_H(t), psi0))))**2

In [18]:
def monte_carlo(n_sample, n_times, t_i, t_f):
    import random
    import itertools
    #prob_index = [(i,j) for i,j in itertools.product(range(n_cavity),range(2))]
    outcome = np.zeros((n_cavity * 2, n_times))
    prob_result = np.zeros((n_cavity * 2))
    dt = (t_f - t_i)/(n_times-1)

    t = t_i
    for z in range(n_times):
        prob_list = [prob(i,j,t).reshape(1)[0] for i,j in itertools.product(range(n_cavity),range(2))]
        prob_result = np.append(prob_result, prob_list)
        prob_line = np.cumsum(prob_list)
        for _ in range(n_sample):
            random_num = random.random()
            for k in range(len(prob_line)):
                if k == 0 and random_num < prob_line[k]:
                    outcome[k, z] += 1
                elif prob_line[k-1] <= random_num and random_num < prob_line[k]:
                    outcome[k, z] += 1
            t += dt
    
    relative_prob = outcome/n_sample

    import pandas as pd
    df_outcome = pd.DataFrame(relative_prob.T)
    df_prob = pd.DataFrame(prob_result)
    df_outcome.index = np.linspace(t_i,t_f,n_sample)
    df_prob.index = np.linspace(t_i,t_f,n_sample)

    return df_outcome, df_prob 

In [19]:
df_outcome, df_prob = monte_carlo(10,10,0,1)

ValueError: Length mismatch: Expected axis has 220 elements, new values have 10 elements

In [8]:
df_outcome

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.111111,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.222222,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.333333,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.444444,0.0,0.0,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.555556,0.0,0.0,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.666667,0.0,0.0,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.777778,0.0,0.0,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.888889,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.000000,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_prob

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.111111,0.0,0.0,0.927685,0.072315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.222222,0.0,0.0,0.732717,0.267283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.333333,0.0,0.0,0.474347,0.525653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.444444,0.0,0.0,0.231095,0.768905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.555556,0.0,0.0,0.076886,0.923114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.666667,0.0,0.0,0.058585,0.941415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.777778,0.0,0.0,0.181754,0.818246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.888889,0.0,0.0,0.408960,0.591040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.000000,0.0,0.0,0.671156,0.328844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
